# COCO Silhouette Dataset Generation Notebook

**Objective:**  
Extract objects (cat, bicycle, backpack, apple) from the COCO dataset using segmentation masks, convert them into silhouettes & composite them onto a blank background to generate a mixed dataset.

**Steps:**
1. Load COCO annotations and images.
2. Filter for the desired object classes.
3. For each selected image, extract segmentation masks and create silhouette cutouts.
4. Composite a random number of silhouettes onto a blank background.
5. Label the composite image based on the total number of objects.

**Reproducibility:**  
A fixed random seed is set to ensure consistent dataset generation.

# Download COCO

In [ ]:
!pip install pycocotools -q

In [ ]:
!mkdir -p coco/images/train2017 coco/annotations

In [ ]:
!wget -q http://images.cocodataset.org/zips/train2017.zip -P coco/images
!unzip -q coco/images/train2017.zip -d coco/images/

In [ ]:
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P coco/annotations
!unzip -q coco/annotations/annotations_trainval2017.zip -d coco/annotations/

# Environment Setup & installation of packages

In [ ]:
import os
import random
import shutil
import torch
import numpy as np
import cv2
from pycocotools.coco import COCO
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
COCO_PATH = "/coco"
ANNOTATION_FILE = os.path.join(COCO_PATH, "/content/coco/annotations/annotations/instances_train2017.json")
IMAGE_FOLDER = os.path.join(COCO_PATH, "train2017")
OUTPUT_FOLDER = "./silhouette_dataset"
TARGET_CLASSES = ["cat", "bicycle", "backpack", "apple"]
MAX_PER_CLASS = 500  # Number of silhouettes per category
IMAGE_SIZE = 128

# Setup & Category Mapping

In [ ]:
coco = COCO(ANNOTATION_FILE)
cat_ids = coco.getCatIds(catNms=TARGET_CLASSES)
cat_map = {cat_id: coco.loadCats(cat_id)[0]['name'] for cat_id in cat_ids}

loading annotations into memory...
Done (t=28.41s)
creating index...
index created!


In [ ]:
target_classes = ["cat", "apple", "bicycle", "backpack"]
cat_map = {cat['id']: cat['name'] for cat in coco.loadCats(coco.getCatIds())}
cat_ids = coco.getCatIds(catNms=target_classes)

In [ ]:
for cid in cat_ids:
    anns = coco.loadAnns(coco.getAnnIds(catIds=[cid]))
    print(f"{cat_map[cid]}: {len(anns)} annotations")


bicycle: 7113 annotations
cat: 4768 annotations
backpack: 8720 annotations
apple: 5851 annotations


# Silhouette Generator

In [ ]:
def generate_silhouette(annotation):
    mask = coco.annToMask(annotation)
    if mask.sum() == 0:
        return None  # skip empty masks
    silhouette = (mask * 255).astype(np.uint8)
    return silhouette

# Main Extraction Loop

In [ ]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for cat_id in tqdm(cat_ids, desc="Processing Categories"):
    anns = coco.loadAnns(coco.getAnnIds(catIds=[cat_id]))
    selected = random.sample(anns, min(MAX_PER_CLASS, len(anns)))

    class_name = cat_map[cat_id].replace(" ", "_")
    class_folder = os.path.join(OUTPUT_FOLDER, class_name)
    os.makedirs(class_folder, exist_ok=True)

    for i, ann in enumerate(selected):
        img_info = coco.loadImgs(ann['image_id'])[0]
        silhouette = generate_silhouette(ann)

        if silhouette is not None:
            resized = cv2.resize(silhouette, (IMAGE_SIZE, IMAGE_SIZE))
            save_path = os.path.join(class_folder, f"{img_info['id']}_{i}.png")
            cv2.imwrite(save_path, resized)
        else:
            print(f"Skipped: ann_id={ann['id']} (empty mask)")

print("Silhouette dataset generation complete.")

Processing Categories: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]

✅ Silhouette dataset generation complete.
